# Use Splinter and/or Beautiful Soup to scrape https://www.dndbeyond.com/classes, and pull out all class names. Store these in a dataframe.


In [101]:
import pandas as pd


In [102]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://bookstoscrape.com'

In [103]:
response = requests.get(url)
response

<Response [200]>

In [104]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [105]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'http://dndbeyond.com/classes'
browser.visit(url) 



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\weird\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [106]:
#I've played Rogue and Fighter. I think I liked the rogue the best.

soup = bs(browser.html, 'html.parser')
all_class = soup.find_all('h3', class_='listing-card__title')
dnd_list = []
for dndclass in all_class:
    dnd_dict = {"class":dndclass.text.strip(' ').strip('\n')}
    dnd_list.append(dnd_dict)
dnd_df =  pd.DataFrame(dnd_list)
dnd_df

,class
0,Barbarian
1,Bard
2,Cleric
3,Druid
4,Fighter
5,Monk
6,Paladin
7,Ranger
8,Rogue
9,Sorcerer


# Now, alter 1 to click on each class name to go to the more detailed page, and pull out the first paragraph about each class. Store these in a dataframe. Your dataframe should have two columns

In [111]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'http://dndbeyond.com/classes'
browser.visit(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\weird\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [114]:
all_classes=[]
#url = 'https://www.dndbeyond.com/classes'
art_url = 'https://www.dndbeyond.com/marketplace/sourcebooks/tashas-cauldron-of-everything'
bh_url = 'https://www.dndbeyond.com/classes/blood-hunter'
#browser.visit(url)
class_dict={}

soup=bs(browser.html, 'html.parser')
div = soup.find_all('div',class_='listing-card__header-primary')


for d in div:
    classes= d.find('h3',class_='listing-card__title').text
    classes= classes.strip()
   
    if classes != 'Artificer' and classes != 'Blood Hunter':
        url = f'https://www.dndbeyond.com/classes/{classes}'
        browser.visit(url)
        soup=bs(browser.html, 'html.parser')
        class_description = soup.find('p', class_='Core-Styles_Core-Body').text
    if classes == 'Artificer':
        browser.visit(art_url)
        soup = bs(browser.html, 'html.parser')
        class_description = soup.find('div',class_='marketplace-item__description j-description').find('p').text
    if classes =='Blood Hunter':
            browser.visit(bh_url)
            soup_page = bs(browser.html, 'html.parser')
            p = soup_page.find('div',class_='content-container').find_all('p')
            class_description = p[3].text    
    class_dict = {'Class':classes, 'Class_Description':class_description}
    all_classes.append(class_dict)
    
    
df = pd.DataFrame(all_classes)

In [115]:
df = df.sort_values('Class')


In [116]:
df
# so it looks like Artificer isn't what we think it should be and I'm guessing it is because we don't 
#actually have access to the book itself. I do in real life, it is in my husband's DND library. But that doesn't help 
#here so we'll get the next best thing. 

,Class,Class_Description
12,Artificer,"With a Master Tier subscription, save your par..."
0,Barbarian,A tall human tribesman strides through a blizz...
1,Bard,Humming as she traces her fingers over an anci...
13,Blood Hunter,"Marred but resolute, his grimacing face drippi..."
2,Cleric,Arms and eyes upraised toward the sun and a pr...
3,Druid,Holding high a gnarled staff wreathed with hol...
4,Fighter,A human in clanging plate armor holds her shie...
5,Monk,Her fists a blur as they deflect an incoming h...
6,Paladin,Clad in plate armor that gleams in the sunligh...
7,Ranger,"Choose a type of favored enemy: aberrations, b..."


# the artificer should say "masters of invention, artificers use ingenuity and magic to unlock extraordinary capabilites in objects." 
